# IRED Trajectory Embedding Analysis

This notebook generates 2D embeddings of IRED optimization trajectories using PCA and nonlinear methods (Isomap) to visualize the manifold structure of the optimization process.

## Data Overview
- 150 matrix inverse problems
- 10 optimization steps per problem (1500 total trajectory points)
- 64-dimensional state vectors (8x8 matrices flattened)
- Energy and error metrics for each step

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.manifold import Isomap
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set style for better plots
plt.style.use('default')
sns.set_palette("husl")

# Create output directories
results_dir = Path('/Users/mkrasnow/Desktop/diff-geo-ired/documentation/results')
figures_dir = Path('/Users/mkrasnow/Desktop/diff-geo-ired/documentation/figures')
results_dir.mkdir(exist_ok=True)
figures_dir.mkdir(exist_ok=True)

print("Imports and directories ready!")

## 1. Load and Preprocess Trajectory Data

In [ ]:
# Load trajectory data
data_path = '/Users/mkrasnow/Desktop/diff-geo-ired/documentation/results/ired_trajectories_raw.npz'
trajectory_data = np.load(data_path, allow_pickle=True)

print("=== Trajectory Data Overview ===")
for key in trajectory_data.keys():
    if hasattr(trajectory_data[key], 'shape'):
        print(f"{key}: {trajectory_data[key].shape} ({trajectory_data[key].dtype})")
    else:
        print(f"{key}: {trajectory_data[key]} ({type(trajectory_data[key])})")

# Extract key arrays
num_problems = int(trajectory_data['num_problems'])
num_steps = int(trajectory_data['num_steps'])
states = trajectory_data['states']  # Shape: (150, 10, 1, 64)
energies = trajectory_data['energies']  # Shape: (150, 10, 1, 1) 
error_metrics = trajectory_data['error_metrics']  # Shape: (150, 10, 1)
landscapes = trajectory_data['landscapes']  # Shape: (150, 10)
steps = trajectory_data['steps']  # Shape: (150, 10)

print(f"\n=== Data Summary ===")
print(f"Number of problems: {num_problems}")
print(f"Steps per problem: {num_steps}")
print(f"Total trajectory points: {num_problems * num_steps}")
print(f"State dimensionality: {states.shape[-1]}")

In [ ]:
# Reshape data for embedding analysis
# Convert from (150, 10, 1, 64) to (1500, 64) for all trajectory points
all_states = states.reshape(-1, states.shape[-1])  # (1500, 64)
all_energies = energies.reshape(-1)  # (1500,)
all_error_metrics = error_metrics.reshape(-1)  # (1500,)

# Create trajectory metadata for visualization
# Track which problem and step each point belongs to
problem_indices = np.repeat(np.arange(num_problems), num_steps)  # (1500,)
step_indices = np.tile(np.arange(num_steps), num_problems)  # (1500,)

# Landscape parameters (for coloring)
all_landscapes = landscapes.reshape(-1)  # (1500,)

print("=== Reshaped Data ===")
print(f"All states: {all_states.shape}")
print(f"All energies: {all_energies.shape}")
print(f"Problem indices: {problem_indices.shape}")
print(f"Step indices: {step_indices.shape}")
print(f"All landscapes: {all_landscapes.shape}")

# Check for any NaN or infinite values
print(f"\n=== Data Quality Check ===")
print(f"States - NaN: {np.isnan(all_states).sum()}, Inf: {np.isinf(all_states).sum()}")
print(f"Energies - NaN: {np.isnan(all_energies).sum()}, Inf: {np.isinf(all_energies).sum()}")
print(f"State range: [{np.min(all_states):.3f}, {np.max(all_states):.3f}]")
print(f"Energy range: [{np.min(all_energies):.3f}, {np.max(all_energies):.3f}]")

## 2. Generate PCA Embedding

In [ ]:
# Apply PCA for linear dimensionality reduction
print("=== Applying PCA Embedding ===")

# Standardize the data
states_standardized = (all_states - np.mean(all_states, axis=0)) / np.std(all_states, axis=0)

# Apply PCA
pca = PCA(n_components=2, random_state=42)
pca_embedding = pca.fit_transform(states_standardized)

print(f"PCA embedding shape: {pca_embedding.shape}")
print(f"Explained variance ratio: {pca.explained_variance_ratio_}")
print(f"Total variance explained: {pca.explained_variance_ratio_.sum():.3f}")

# Save PCA embedding
pca_save_path = results_dir / 'pca_embedding.npy'
np.save(pca_save_path, pca_embedding)
print(f"Saved PCA embedding to: {pca_save_path}")

## 3. Generate Isomap Embedding

In [ ]:
# Apply Isomap for nonlinear dimensionality reduction
print("=== Applying Isomap Embedding ===")

# Use a reasonable number of neighbors based on data size
n_neighbors = min(15, max(5, int(np.sqrt(len(all_states)))))
print(f"Using n_neighbors = {n_neighbors}")

# Apply Isomap
isomap = Isomap(n_components=2, n_neighbors=n_neighbors, path_method='D')
isomap_embedding = isomap.fit_transform(states_standardized)

print(f"Isomap embedding shape: {isomap_embedding.shape}")
print(f"Reconstruction error: {isomap.reconstruction_error():.6f}")

# Save Isomap embedding
isomap_save_path = results_dir / 'isomap_embedding.npy'
np.save(isomap_save_path, isomap_embedding)
print(f"Saved Isomap embedding to: {isomap_save_path}")

## 4. Create Trajectory Visualizations

In [ ]:
def create_trajectory_plot(embedding, title, filename, n_trajectories_show=10):
    """
    Create trajectory visualization showing temporal progression
    """
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 5))
    
    # Plot 1: All points colored by step index
    scatter = ax1.scatter(embedding[:, 0], embedding[:, 1], 
                         c=step_indices, cmap='viridis', alpha=0.6, s=20)
    ax1.set_title(f'{title} - Colored by Step Index')
    ax1.set_xlabel('Component 1')
    ax1.set_ylabel('Component 2')
    plt.colorbar(scatter, ax=ax1, label='Optimization Step')
    
    # Plot 2: All points colored by energy
    scatter2 = ax2.scatter(embedding[:, 0], embedding[:, 1], 
                          c=all_energies, cmap='plasma', alpha=0.6, s=20)
    ax2.set_title(f'{title} - Colored by Energy')
    ax2.set_xlabel('Component 1')
    ax2.set_ylabel('Component 2')
    plt.colorbar(scatter2, ax=ax2, label='Energy')
    
    # Plot 3: Individual trajectories (subset)
    colors = plt.cm.tab10(np.linspace(0, 1, n_trajectories_show))
    
    for i in range(min(n_trajectories_show, num_problems)):
        # Get points for this trajectory
        traj_mask = problem_indices == i
        traj_points = embedding[traj_mask]
        
        # Plot trajectory line
        ax3.plot(traj_points[:, 0], traj_points[:, 1], 
                color=colors[i], alpha=0.7, linewidth=1.5, 
                label=f'Problem {i}' if i < 5 else '')
        
        # Mark start and end points
        ax3.scatter(traj_points[0, 0], traj_points[0, 1], 
                   color=colors[i], marker='o', s=50, edgecolor='white', linewidth=1)
        ax3.scatter(traj_points[-1, 0], traj_points[-1, 1], 
                   color=colors[i], marker='s', s=50, edgecolor='white', linewidth=1)
    
    ax3.set_title(f'{title} - Individual Trajectories')
    ax3.set_xlabel('Component 1')
    ax3.set_ylabel('Component 2')
    if n_trajectories_show <= 5:
        ax3.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    
    plt.tight_layout()
    
    # Save the plot
    save_path = figures_dir / filename
    plt.savefig(save_path, dpi=300, bbox_inches='tight')
    plt.show()
    
    print(f"Saved plot to: {save_path}")
    
    return fig

In [ ]:
# Create PCA visualization
pca_fig = create_trajectory_plot(
    pca_embedding, 
    'PCA Embedding of IRED Trajectories', 
    'pca_trajectories_matrix_inverse.png'
)

In [ ]:
# Create Isomap visualization
isomap_fig = create_trajectory_plot(
    isomap_embedding, 
    'Isomap Embedding of IRED Trajectories', 
    'isomap_trajectories_matrix_inverse.png'
)

## 5. Detailed Trajectory Analysis

In [ ]:
def analyze_embedding_properties(embedding, name):
    """
    Analyze geometric properties of the embedding
    """
    print(f"\n=== {name} Embedding Analysis ===")
    
    # Basic statistics
    print(f"Embedding range: X=[{embedding[:, 0].min():.3f}, {embedding[:, 0].max():.3f}], "
          f"Y=[{embedding[:, 1].min():.3f}, {embedding[:, 1].max():.3f}]")
    
    # Compute trajectory lengths in embedding space
    trajectory_lengths = []
    initial_distances = []
    final_distances = []
    
    for i in range(num_problems):
        traj_mask = problem_indices == i
        traj_points = embedding[traj_mask]
        
        # Compute path length
        diffs = np.diff(traj_points, axis=0)
        distances = np.linalg.norm(diffs, axis=1)
        total_length = np.sum(distances)
        trajectory_lengths.append(total_length)
        
        # Displacement from start to end
        displacement = np.linalg.norm(traj_points[-1] - traj_points[0])
        initial_distances.append(displacement)
        
        # Distance from origin
        final_distance = np.linalg.norm(traj_points[-1])
        final_distances.append(final_distance)
    
    trajectory_lengths = np.array(trajectory_lengths)
    initial_distances = np.array(initial_distances)
    final_distances = np.array(final_distances)
    
    print(f"Trajectory path lengths: mean={trajectory_lengths.mean():.3f}, std={trajectory_lengths.std():.3f}")
    print(f"Start-to-end displacement: mean={initial_distances.mean():.3f}, std={initial_distances.std():.3f}")
    print(f"Final distances from origin: mean={final_distances.mean():.3f}, std={final_distances.std():.3f}")
    
    # Compute trajectory sinuosity (path length / displacement)
    sinuosity = trajectory_lengths / (initial_distances + 1e-8)  # Avoid division by zero
    print(f"Trajectory sinuosity: mean={sinuosity.mean():.3f}, std={sinuosity.std():.3f}")
    
    return {
        'trajectory_lengths': trajectory_lengths,
        'displacements': initial_distances,
        'final_distances': final_distances,
        'sinuosity': sinuosity
    }

# Analyze both embeddings
pca_analysis = analyze_embedding_properties(pca_embedding, "PCA")
isomap_analysis = analyze_embedding_properties(isomap_embedding, "Isomap")

In [ ]:
# Compare embedding properties
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

# Trajectory lengths comparison
axes[0, 0].hist(pca_analysis['trajectory_lengths'], bins=20, alpha=0.6, label='PCA', density=True)
axes[0, 0].hist(isomap_analysis['trajectory_lengths'], bins=20, alpha=0.6, label='Isomap', density=True)
axes[0, 0].set_xlabel('Trajectory Length')
axes[0, 0].set_ylabel('Density')
axes[0, 0].set_title('Distribution of Trajectory Lengths')
axes[0, 0].legend()

# Displacement comparison
axes[0, 1].hist(pca_analysis['displacements'], bins=20, alpha=0.6, label='PCA', density=True)
axes[0, 1].hist(isomap_analysis['displacements'], bins=20, alpha=0.6, label='Isomap', density=True)
axes[0, 1].set_xlabel('Start-to-End Displacement')
axes[0, 1].set_ylabel('Density')
axes[0, 1].set_title('Distribution of Trajectory Displacements')
axes[0, 1].legend()

# Sinuosity comparison
axes[1, 0].hist(pca_analysis['sinuosity'], bins=20, alpha=0.6, label='PCA', density=True)
axes[1, 0].hist(isomap_analysis['sinuosity'], bins=20, alpha=0.6, label='Isomap', density=True)
axes[1, 0].set_xlabel('Sinuosity (Path Length / Displacement)')
axes[1, 0].set_ylabel('Density')
axes[1, 0].set_title('Distribution of Trajectory Sinuosity')
axes[1, 0].legend()

# Energy vs trajectory length
final_energies = []
for i in range(num_problems):
    traj_mask = problem_indices == i
    final_energies.append(all_energies[traj_mask][-1])

final_energies = np.array(final_energies)

axes[1, 1].scatter(pca_analysis['trajectory_lengths'], final_energies, alpha=0.6, label='PCA')
axes[1, 1].set_xlabel('PCA Trajectory Length')
axes[1, 1].set_ylabel('Final Energy')
axes[1, 1].set_title('Trajectory Length vs Final Energy')

plt.tight_layout()
plt.savefig(figures_dir / 'embedding_analysis_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print(f"Saved comparison plot to: {figures_dir / 'embedding_analysis_comparison.png'}")

## 6. Save Analysis Results

Save embedding coordinates and analysis data for future use.

In [ ]:
# Save comprehensive analysis results
analysis_data = {
    # Embedding coordinates
    'pca_embedding': pca_embedding,
    'isomap_embedding': isomap_embedding,
    
    # Metadata for each point
    'problem_indices': problem_indices,
    'step_indices': step_indices,
    'all_energies': all_energies,
    'all_error_metrics': all_error_metrics,
    'all_landscapes': all_landscapes,
    
    # Trajectory analysis results
    'pca_trajectory_lengths': pca_analysis['trajectory_lengths'],
    'isomap_trajectory_lengths': isomap_analysis['trajectory_lengths'],
    'pca_displacements': pca_analysis['displacements'],
    'isomap_displacements': isomap_analysis['displacements'],
    'pca_sinuosity': pca_analysis['sinuosity'],
    'isomap_sinuosity': isomap_analysis['sinuosity'],
    
    # PCA parameters
    'pca_explained_variance_ratio': pca.explained_variance_ratio_,
    'pca_total_variance_explained': pca.explained_variance_ratio_.sum(),
    
    # Isomap parameters
    'isomap_n_neighbors': n_neighbors,
    'isomap_reconstruction_error': isomap.reconstruction_error(),
    
    # Data parameters
    'num_problems': num_problems,
    'num_steps': num_steps,
    'state_dimensionality': states.shape[-1]
}

# Save to NPZ file
analysis_save_path = results_dir / 'ired_embedding_analysis.npz'
np.savez_compressed(analysis_save_path, **analysis_data)

print(f"Saved comprehensive analysis to: {analysis_save_path}")
print(f"\nFiles created:")
print(f"- {results_dir / 'pca_embedding.npy'}")
print(f"- {results_dir / 'isomap_embedding.npy'}")
print(f"- {results_dir / 'ired_embedding_analysis.npz'}")
print(f"- {figures_dir / 'pca_trajectories_matrix_inverse.png'}")
print(f"- {figures_dir / 'isomap_trajectories_matrix_inverse.png'}")
print(f"- {figures_dir / 'embedding_analysis_comparison.png'}")

print("\n=== Task 5.1 Completion Summary ===")
print("✅ Generated PCA and Isomap 2D embeddings")
print("✅ Created trajectory visualizations with temporal progression")
print("✅ Saved embedding coordinates in NPY format")
print("✅ Analyzed geometric properties of trajectories")
print("✅ Generated publication-quality figures")
print(f"\nData ready for geometric analysis (Task 5.2)!")

## 7. Trajectory Geometric Properties Analysis

Compute detailed path lengths and discrete curvatures for each trajectory in both embedding spaces.

In [ ]:
# Import the detailed geometric analysis module
import importlib.util
import sys

spec = importlib.util.spec_from_file_location("compute_detailed_trajectory_geometry", "compute_detailed_trajectory_geometry.py")
geom_module = importlib.util.module_from_spec(spec)
sys.modules["compute_detailed_trajectory_geometry"] = geom_module
spec.loader.exec_module(geom_module)

# Use the existing embedding data to compute geometric properties
print("Computing geometric properties for trajectory embeddings...")

# Analyze both embeddings
pca_summary, pca_detailed = geom_module.analyze_trajectory_geometry_detailed(
    pca_embedding, problem_indices, step_indices, "PCA"
)

isomap_summary, isomap_detailed = geom_module.analyze_trajectory_geometry_detailed(
    isomap_embedding, problem_indices, step_indices, "Isomap"
)

print(f"
Geometric analysis complete\!")
print(f"Summary records: {len(pca_summary) + len(isomap_summary)}")
print(f"Detailed curvature records: {len(pca_detailed) + len(isomap_detailed)}")

In [ ]:
# Combine results and save to CSV files as specified in Task 5.2
import pandas as pd

# Combine summary results (path lengths)
all_summary_results = pca_summary + isomap_summary
lengths_df = pd.DataFrame(all_summary_results)

# Save path length results
lengths_csv_path = results_dir / "ired_trajectory_lengths.csv"
lengths_df.to_csv(lengths_csv_path, index=False)
print(f"✅ Saved trajectory lengths to: {lengths_csv_path}")

# Combine detailed curvature results
all_curvature_results = pca_detailed + isomap_detailed
curvatures_df = pd.DataFrame(all_curvature_results)

# Save curvature results
curvatures_csv_path = results_dir / "ired_trajectory_curvatures.csv"
curvatures_df.to_csv(curvatures_csv_path, index=False)
print(f"✅ Saved trajectory curvatures to: {curvatures_csv_path}")

# Display sample data
print(f"
=== Trajectory Lengths CSV Preview ===")
print(lengths_df.head(10))
print(f"
Shape: {lengths_df.shape}")

print(f"
=== Trajectory Curvatures CSV Preview ===")
print(curvatures_df.head(10))
print(f"
Shape: {curvatures_df.shape}")

## Task 5.2 Completion Summary

Extended the trajectory embedding analysis with detailed geometric property calculations.

In [ ]:
print("
" + "="*60)
print("🎯 TASK 5.2 GEOMETRIC PROPERTIES ANALYSIS COMPLETE")
print("="*60)

print("
✅ COMPLETED OBJECTIVES:")
print("  • Extended ired_embedding_analysis.ipynb with geometric analysis")
print("  • Computed path lengths using numpy.linalg.norm for distance calculations")
print("  • Computed discrete curvatures with epsilon=1e-8 regularization")
print("  • Handled division by zero in curvature calculation")
print("  • Skipped endpoints for curvature computation (interior points only)")
print("  • Saved results in CSV format for easy analysis")

print(f"
📁 OUTPUT FILES CREATED:")
print(f"  • {lengths_csv_path}")
print(f"  • {curvatures_csv_path}")

print(f"
📊 DATA SUMMARY:")
print(f"  • Trajectories analyzed: {len(lengths_df)//2} per embedding ({len(lengths_df)} total records)")
print(f"  • Embedding types: {', '.join(lengths_df["embedding_type"].unique())}")
print(f"  • Curvature measurements: {len(curvatures_df)} interior points")

for emb_type in ["PCA", "Isomap"]:
    subset = lengths_df[lengths_df["embedding_type"] == emb_type]
    avg_length = subset["path_length"].mean()
    avg_curvature = subset["mean_curvature"].mean()
    print(f"  • Average path length ({emb_type}): {avg_length:.4f}")
    print(f"  • Average mean curvature ({emb_type}): {avg_curvature:.6f}")

print(f"
🔍 TECHNICAL IMPLEMENTATION:")
print(f"  • Used numpy.linalg.norm for robust distance calculations")
print(f"  • Applied epsilon regularization (ε = 1e-8) for numerical stability") 
print(f"  • Computed curvature only for interior points (excludes endpoints)")
print(f"  • Discrete curvature formula: κ = |y_{{t+1}} - 2y_t + y_{{t-1}}| / |y_{{t+1}} - y_t|²")
print(f"  • CSV output format optimized for downstream analysis")

print(f"
✨ Task 5.2 complete\! Geometric properties computed and saved.")
print("="*60)